#3.YAPAY SİNİR AĞI TASARLAYARAK MODELİN PERFORMANSINI DEĞERLENDİRME VE TEST VERİSİNİN ACCURACY, PRECISION, RECALL VE F1 SKORU, KARMAŞIKLIK MATRİSİ, CLASSIFICATION REPORT'UNU GÖSTERME

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import tensorflow as tf
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Dropout
from keras.models import Sequential
from keras.optimizers import Adam

In [3]:
print(os.listdir("/gdrive/MyDrive/Kidney_Cancer/Kidney Cancer"))

DATADIR = '/gdrive/MyDrive/Kidney_Cancer/Kidney Cancer'

['Tumor', 'Normal']


In [4]:
CATEGORIES = ["Tumor", "Normal"]
IMG_SIZE = 100

In [5]:
all_data = []
for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    class_num = CATEGORIES.index(category)
    for img in os.listdir(path):
        try:
            img_array = cv2.imread(os.path.join(path, img))
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
            all_data.append([new_array, class_num])
        except Exception as e:
            pass

In [6]:
print("Toplam Görüntü Sayısı: ", len(all_data))

Toplam Görüntü Sayısı:  10000


In [7]:
X = []
y = []

for features, label in all_data:
    X.append(features)
    y.append(label)

X = np.array(X)
y = np.array(y)


In [8]:

# Veriyi normalize etme
X = X / 255.0

In [9]:
# Train-test split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state =42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [10]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [11]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(7200, 100, 100, 3)
(2000, 100, 100, 3)
(800, 100, 100, 3)
(7200,)
(2000,)
(800,)


In [12]:
train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=2)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=2)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=2)

In [13]:
# Yapay Sinir Ağı Modeli
model_nn = Sequential()
model_nn.add(Flatten(input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model_nn.add(Dense(256, activation='relu'))
model_nn.add(Dropout(0.4))
model_nn.add(Dense(128, activation='relu'))
model_nn.add(Dropout(0.4))
model_nn.add(Dense(2, activation='softmax'))

In [14]:
# Modeli derleme
model_nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
from tensorflow import keras

callback_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model.h5',
        monitor = 'val_accuracy', save_best_only=True, verbose=3
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=3)
]

In [16]:
# Modeli eğitme
history = model_nn.fit(x_train, train_yCl,
                    batch_size=64,
                    validation_data = (x_val, valid_yCl),
                    callbacks = callback_list,
                    epochs = 30
                    )

Epoch 1/30
113/113 [==============================] - ETA: 0s - loss: 0.4597 - accuracy: 0.8497
Epoch 1: val_accuracy improved from -inf to 0.97625, saving model to model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


113/113 [==============================] - 19s 144ms/step - loss: 0.4597 - accuracy: 0.8497 - val_loss: 0.0871 - val_accuracy: 0.9762
Epoch 2/30
113/113 [==============================] - ETA: 0s - loss: 0.0893 - accuracy: 0.9708
Epoch 2: val_accuracy improved from 0.97625 to 0.98750, saving model to model.h5
113/113 [==============================] - 17s 147ms/step - loss: 0.0893 - accuracy: 0.9708 - val_loss: 0.0382 - val_accuracy: 0.9875
Epoch 3/30
113/113 [==============================] - ETA: 0s - loss: 0.0607 - accuracy: 0.9796
Epoch 3: val_accuracy improved from 0.98750 to 0.99750, saving model to model.h5
113/113 [==============================] - 16s 139ms/step - loss: 0.0607 - accuracy: 0.9796 - val_loss: 0.0175 - val_accuracy: 0.9975
Epoch 4/30
113/113 [==============================] - ETA: 0s - loss: 0.0605 - accuracy: 0.9772
Epoch 4: val_accuracy did not improve from 0.99750
113/113 [==============================] - 16s 139ms/step - loss: 0.0605 - accuracy: 0.9772 - val

In [17]:
# Modelin performansını değerlendirme
score_nn_val_transfer = model_nn.evaluate(x_val, valid_yCl)
print("Validation Accuracy (Neural Network): {:.2f}%".format(score_nn_val_transfer[1] * 100))

score_nn_test_transfer = model_nn.evaluate(x_test, test_yCl)
print("Test Accuracy (Neural Network): {:.2f}%".format(score_nn_test_transfer[1] * 100))

score_nn_train_transfer = model_nn.evaluate(x_train, train_yCl)
print("Train Accuracy (Neural Network): {:.2f}%".format(score_nn_train_transfer[1]))


25/25 [==============================] - 1s 39ms/step - loss: 0.0146 - accuracy: 0.9962
Validation Accuracy (Neural Network): 99.62%
63/63 [==============================] - 2s 35ms/step - loss: 0.0199 - accuracy: 0.9930
Test Accuracy (Neural Network): 99.30%
225/225 [==============================] - 5s 24ms/step - loss: 6.6020e-04 - accuracy: 0.9999
Train Accuracy (Neural Network): 1.00%


In [18]:
# Tahmin yapma
y_pred_nn = model_nn.predict(x_test)
y_pred_classes_nn = np.argmax(y_pred_nn, axis=1)


63/63 [==============================] - 2s 29ms/step


In [19]:

# Accuracy, Precision, Recall, F1 Score
accuracy_nn = accuracy_score(y_test, y_pred_classes_nn)
precision_nn = precision_score(y_test, y_pred_classes_nn)
recall_nn = recall_score(y_test, y_pred_classes_nn)
f1_nn = f1_score(y_test, y_pred_classes_nn)

print("Neural Network Test Accuracy: {:.2f}".format(accuracy_nn))
print("Neural Network Test Precision: {:.2f}".format(precision_nn))
print("Neural Network Test Recall: {:.2f}".format(recall_nn))
print("Neural Network Test F1 Score: {:.2f}".format(f1_nn))

# Karmaşıklık Matrisi
conf_matrix_nn = confusion_matrix(y_test, y_pred_classes_nn)
print("Neural Network Confusion Matrix:\n", conf_matrix_nn)

# Classification Report
class_report_nn = classification_report(y_test, y_pred_classes_nn)
print("Neural Network Classification Report:\n", class_report_nn)


Neural Network Test Accuracy: 0.99
Neural Network Test Precision: 0.99
Neural Network Test Recall: 1.00
Neural Network Test F1 Score: 0.99
Neural Network Confusion Matrix:
 [[998  14]
 [  0 988]]
Neural Network Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      0.99      1012
           1       0.99      1.00      0.99       988

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000

